In [ ]:
import itertools

import matplotlib.pyplot as plt
import numpy as np

from stats import *
from utils import *
import visual as vi
from Inlier_Thresholder import Inlier_Thresholder
import scipy.io as spi
from time import time
import os
import seaborn as sns
from itertools import chain

In [ ]:
directory = 'C:/Users/carlo/IACV PROJECT/adelFM'

# List all files in the directory
files = os.listdir(directory)

# Filter out only the .mat files
names = [file for file in files if file.endswith('.mat')]

# Load each .mat file
mat_data = {}
for file in names:
    file_path = os.path.join(directory, file)
    mat_data[file] = spi.loadmat(file_path)
    

# 1) INLIER THRESHOLDS ESTIMATION

The plots of the silhouette scores are done only if some outliers are present, otherwise a simple plot with the threshold is displayed

### 1a) SED

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)

thresholds = [[] for i in range(number_of_data_to_analyse)]

for i in range(number_of_data_to_analyse):
    print(i)
    data = mat_data[names[i]]
    res, n_inl, n_outl = compute_inliers_residual_curve(data, type='FM', return_inl_outl=True, verbose=False, metric="sed")

    for j in range(len(res)):
        anomaly_detector=Inlier_Thresholder(res[j], n_inl[j], n_outl[j],  alphas={"Median AD": 4 , "Variance based": 5 ,"Rosseeuw SN":4,"Rosseeuw QN":4 })

        labels, threshold=anomaly_detector.compute_inlier_threshold("Variance based") #use_best_method()  # use best method among the statistical ones, best according to silhouette
        #print("eccolo: ", threshold)
        thresholds[i].append(threshold)
        
        if len(np.unique(labels))!=1:
            silhouette_scr, silhouette_avg, values = silhouette_score_and_average(res[j], labels)
            plot_silhouette(silhouette_scr, labels, silhouette_avg, values, -thresholds[i][j])
            
        else:
            plt.figure()
            plt.scatter(np.arange(len(res[j])),-np.sort(res[j]), marker='.', s=30, lw=0, alpha=0.7)
            plt.axhline(y=-thresholds[i][j], color="purple" , linestyle="--")
            plt.show()

  

### 1b) SAMPSON

In [ ]:
# input parameter, how many files to analyse
number_of_data_to_analyse=len(names)

thresholds_1 = [[] for i in range(number_of_data_to_analyse)]

for i in range(number_of_data_to_analyse):
    print(i)
    data = mat_data[names[i]]
    res, n_inl, n_outl = compute_inliers_residual_curve(data, type='FM', return_inl_outl=True, verbose=False, metric="sampson")

    for j in range(len(res)):
        anomaly_detector=Inlier_Thresholder(res[j], n_inl[j], n_outl[j],  alphas={"Median AD": 4 , "Variance based": 5 ,"Rosseeuw SN":4,"Rosseeuw QN":4 })

        labels, threshold=anomaly_detector.compute_inlier_threshold("Variance based") #use_best_method()  # use best method among the statistical ones, best according to silhouette
        #print("eccolo: ", threshold)
        thresholds_1[i].append(threshold)
        
        if len(np.unique(labels))!=1:
            silhouette_scr, silhouette_avg, values = silhouette_score_and_average(res[j], labels)
            plot_silhouette(silhouette_scr, labels, silhouette_avg, values, -thresholds_1[i][j])
            
        else:
            plt.figure()
            plt.scatter(np.arange(len(res[j])),-np.sort(res[j]), marker='.', s=30, lw=0, alpha=0.7)
            plt.axhline(y=-thresholds_1[i][j], color="purple" , linestyle="--")
            plt.show()
  

# 2) ENSEMBLE 

In [ ]:
ens_labels_1=[]
for i in range(len(thresholds_1)):
    print(i)
    data=mat_data[names[i]]
    ens = build_ensemble_mask(data, plot=False, verbose=False, threshold=thresholds[i])
    ens1 = build_ensemble_mask(data, plot=False, verbose=False, threshold=thresholds_1[i])
    
    im1=data["img1"]
    im2=data["img2"]
    
    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

    points=extract_points(models,data)

    tot_src=points[1]
    tot_dst=points[2]

    points=points[0]
    
    image_lab=[]
    
    for j in range(len(models)):
        
    
        ensemble=np.where(ens[j]+ens1[j]<2,0,1)
        image_lab.append(ensemble)
        



        src = points["src_points"][j][np.where(ensemble==0)]
        dst = points["dst_points"][j][np.where(ensemble==0)]
        plt.figure()
        draw_matches(im1,im2,src,dst)
        plt.show()
    ens_labels_1.append(image_lab)

# 3) INFLUENCE FUNCTION SCORE OF THE RESULTS

In [ ]:

## outlier scores for a single model of a single image
def outlier_score(src , dst):
    
    ## initial quantities
    N=src.shape[0]
    
    ## fit the baseline
    FM,baseline_mask=cv2.findFundamentalMat(src, dst, method=cv2.FM_8POINT)#verify_pygcransac_H(src,dst,img1,img2, threshold, verbose=False)
    
    ## initialize the return quantity
    outlier_scores_FM=np.zeros(N)
    
    
    ## loop through all the points
    for i in range(N):
        src_i = np.delete(src, i, axis=0)
        dst_i = np.delete(dst, i, axis=0)
        
        ## fit the model
        FM_i,mask_i=cv2.findFundamentalMat(src_i, dst_i, method=cv2.FM_8POINT)#verify_pygcransac_H(src_i,dst_i,img1,img2, threshold, verbose=False)

        ## outlier score
        outlier_scores_FM[i]=sum(sum((FM-FM_i)**2))
        
    ## return
    return outlier_scores_FM

In [ ]:
outlier_scores_FM=[]

for i in range(number_of_data_to_analyse):
    data=mat_data[names[i]]

    img1, img2 = data["img1"], data["img2"]

    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

    points=extract_points(models,data)

    tot_src=points[1]
    tot_dst=points[2]

    points=points[0]

    out_score_img_i_FM=[]

    for m in range(len(models)):


        src = points["src_points"][m]
        dst = points["dst_points"][m]


        out_score_img_i_FM.append(outlier_score(src,dst))

    outlier_scores_FM.append(out_score_img_i_FM)

In [ ]:
ratios=[]

for j in range(number_of_data_to_analyse):
    image_ratio=[]

    for i in range(len(outlier_scores_FM[j])):
         print(f"img{j} model{i}")
         print(f"inliers avarage score ({sum(ens_labels_1[j][i])}): ",np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==1)]),f" Outliers avarage score ({len(ens_labels_1[j][i])-(sum(ens_labels_1[j][i]))}): " ,
               np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==0)]), f"Factor of the two: ", np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==0)])/np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==1)]))
        
         image_ratio.append(np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==0)])/np.mean(outlier_scores_FM[j][i][np.where(ens_labels_1[j][i]==1)]))
        
    ratios.append(image_ratio)

# 4) Correct only the immages for which it is worth it (high influence function ratio)

In [ ]:
useless_corrections=[]

final_labels=[]
print("These are the images and models for which the correction is not worth it: \n")
for i in range(number_of_data_to_analyse):
    image_labels=[]
    
    for j in range(len(outlier_scores_FM[i])):
        model_labels=[]
        if ratios[i][j]<30:
            print(f"Image {i} Model {j}")
            useless_corrections.append((i,j))
            
            model_labels=np.ones(len(ens_labels_1[i][j])).astype(int)
            
        else:
            model_labels=ens_labels_1[i][j]
            
        image_labels.append(model_labels)
        
    
    final_labels.append(image_labels)     
            
            #ens_labels_1[i][j]=np.ones(len(ens_labels_1[i][j])).astype(int)
            
            

In [ ]:
for i in range(len(thresholds)):
    print(i)
    data=mat_data[names[i]]

    
    im1=data["img1"]
    im2=data["img2"]
    
    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

    points=extract_points(models,data)

    tot_src=points[1]
    tot_dst=points[2]

    points=points[0]
    
    image_lab=[]
    
    
    for j in range(len(models)):
        
        if (i,j) not in useless_corrections:
            lab=final_labels[i][j]
        



            src = points["src_points"][j][np.where(lab==0)]
            dst = points["dst_points"][j][np.where(lab==0)]
            plt.figure()
            draw_matches(im1,im2,src,dst)
            plt.show()
            
        else:
            src = np.array([])#points["src_points"][j]
            dst = np.array([])# points["dst_points"][j]
            plt.figure()
            draw_matches(im1,im2,src,dst)
            plt.show()
            
        


influence_labels=[]

for j in range(number_of_data_to_analyse):
    
    image_labels=[]

    for i in range(len(outlier_scores_FM[j])):
        #m=(outlier_scores_FM[j][i]>=np.median(outlier_scores_FM[j][i])+2*np.std(outlier_scores_FM[j][i]))
        anomaly_detector=Inlier_Thresholder(outlier_scores_FM[j][i],  alphas={"Median AD": 5 , "Variance based": 1.2 ,"Rosseeuw SN":1.5,"Rosseeuw QN":5 })
        
        
        labels, threshold=anomaly_detector.compute_inlier_threshold("Variance based")  # use best method among the statistical ones, best according to silhouette

        #labels 1 outlier 0 inlier
        m=abs(labels-1)
        
        image_labels.append(m)
        if len(np.unique(m))==2: 
            silhouette_scr, silhouette_avg, values = silhouette_score_and_average(outlier_scores_FM[j][i], m)
        
       # m=abs(m.astype(int)-1)
            plot_silhouette(silhouette_scr, m, silhouette_avg=silhouette_avg, values=values, threhsold=-threshold)

        data=mat_data[names[j]]
        im1=data["img1"]
        im2=data["img2"]

        outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

        points=extract_points(models,data)

        tot_src=points[1]
        tot_dst=points[2]

        points=points[0]

        src = points["src_points"][i][np.where(m==0)]
        dst = points["dst_points"][i][np.where(m==0)]
        plt.figure()
        draw_matches(im1,im2,src,dst)
        plt.show()
        
    influence_labels.append(image_labels)

ens_labels_influence_ensemble=[]
for i in range(len(thresholds)):
    print(i)
    data=mat_data[names[i]]

    
    im1=data["img1"]
    im2=data["img2"]
    
    outliers, models = vi.group_models(data)["outliers"], vi.group_models(data)["models"]

    points=extract_points(models,data)

    tot_src=points[1]
    tot_dst=points[2]

    points=points[0]
    
    image_lab=[]
    
    
    for j in range(len(models)):
        
    
        ensemble=np.where(influence_labels[i][j]+ens_labels_1[i][j]<1,0,1)
        image_lab.append(ensemble)
        



        src = points["src_points"][j][np.where(ensemble==0)]
        dst = points["dst_points"][j][np.where(ensemble==0)]
        plt.figure()
        draw_matches(im1,im2,src,dst)
        plt.show()
        
    ens_labels_influence_ensemble.append(image_lab)
